### Посмотрим на данные

In [1]:
from PIL import Image

In [2]:
import cv2
import torch

In [9]:
path = '/kaggle/input/semantic-drone-dataset/dataset/semantic_drone_dataset/'

In [13]:
orig_img = Image.open(os.path.join(path,"original_images/000.jpg"))
orig_img

In [15]:
mask_img = Image.open(os.path.join(path, "label_images_semantic/000.png"))
mask_img

In [16]:
import numpy as np
mask_img_arr = np.array(mask_img)
mask_img_arr.shape

In [17]:
np.unique(mask_img_arr)

### Создадим датасет

In [18]:
from torch.utils.data import Dataset, DataLoader
import os

class SegmentationDataset(Dataset):
    
    def __init__(self, imgs_dir, masks_dir):
        self.imgs_dir = imgs_dir
        self.masks_dir = masks_dir
        
        self.imgs_paths = os.listdir(self.imgs_dir)
        self.imgs_paths.sort()
        
        self.masks_paths = os.listdir(self.masks_dir)
        self.masks_paths.sort()
        
    def __len__(self):
        return len(self.imgs_paths)
    
    def __getitem__(self, idx):
        img = cv2.imread(os.path.join(self.imgs_dir, self.imgs_paths[idx]))
        # TODO: Изменить размер изображения, используя Pytorch Transformations, вынести размер изображения как параметр
        img = cv2.resize(img, (512, 512))
        # TODO: Перевести в тензор, используя Pytorch Transformations
        img = torch.from_numpy(img).float()
        # TODO: Добавить нормировку изображения, используя Pytorch Transformations
        # TODO: Добавить аугментацию
        
        # Меняем размерность с (ширина x высота x количество каналов) на (количество каналов х ширина х высота)
        img = img.permute(2, 0, 1)
        
        mask = cv2.imread(os.path.join(self.masks_dir, self.masks_paths[idx]), cv2.IMREAD_GRAYSCALE)
        # TODO: Аналогично совершить эти преобразования, используя Pytorch Transformations
        mask = cv2.resize(mask, (512, 512))
        mask = torch.from_numpy(mask).long()
        
        return img, mask

In [21]:
dataset = SegmentationDataset(os.path.join(path, 'original_images/'),
                                                 os.path.join(path, "label_images_semantic/"))

In [22]:
# TODO: Отдельный датасет на validation set
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [300, 100])

In [23]:
len(train_dataset)

In [24]:
img, mask = next(iter(train_dataset))

In [25]:
img.shape, img.dtype

In [26]:
mask.shape, mask.dtype

In [49]:
# TODO: Поиграться с размером батча
batch_size= 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# TODO: Отдельный loader на validation set
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

### Создадим модель

In [30]:
!pip install segmentation_models_pytorch

In [31]:
import segmentation_models_pytorch as smp

model = smp.Unet('mobilenet_v2', encoder_weights='imagenet', classes=23)

In [32]:
model

In [33]:
# TODO: Заменить голову на свою :)
# model.segmentation_head = ....
model.segmentation_head

In [34]:
# TODO: Поиграться с заморозкой весов
for param in model.encoder.parameters():
    param.requires_grad = False

In [35]:
img_batch, mask_batch = next(iter(train_loader))

In [36]:
img_batch.shape

In [37]:
mask_batch.shape

In [38]:
output_batch = model(img_batch)

In [39]:
output_batch.shape

In [40]:
from torch.nn import CrossEntropyLoss

CrossEntropyLoss()(output_batch, mask_batch) 

### Обучение

In [41]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('Using device:', device)

In [42]:
model.to(device)

In [43]:
epoch_count = 10

loss = CrossEntropyLoss()

In [44]:
# TODO: поиграться с learning rate
learning_rate = 0.01

# TODO поиграться с выбором алгоритма в целом
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [45]:
from sklearn.metrics import accuracy_score

In [51]:
from tqdm import tqdm

min_val_loss = np.inf
val_loss_epoch = []
best_model = None

train_loss_history = []
val_loss_history = []
val_acc_history = []

for epoch_num in range(epoch_count):
    for batch_num, (img_batch, mask_batch) in enumerate(tqdm(train_loader)):

        img_batch = img_batch.to(device) 
        mask_batch = mask_batch.to(device)
    
        optimizer.zero_grad()
        output_batch = model(img_batch)
        loss_value = loss(output_batch, mask_batch)
        loss_value.backward()
        optimizer.step()           
     
        # TODO: Добавить вычисление метрики, исходя из значений output_batch и mask_batch
        # Варианта метрик: IoU, попиксельная Accuracy
    print(f"Epoch {epoch_num} / {epoch_count} | train loss = {loss_value}")    
        
#     #TODO: Добавить расчет метрик и лосса на валидации и их логирование
    val_losses = []
    val_accs = []
    for batch_num, (img_batch, mask_batch) in enumerate(tqdm(val_loader)):

        img_batch = img_batch.to(device) 
        mask_batch = mask_batch.to(device)
    
        output_batch = model(img_batch)
        val_losses.append(loss(output_batch, mask_batch).item())
        val_accs.append(accuracy_score(np.argmax(output_batch.cpu().detach().numpy(), axis=1).reshape(-1),
                                       mask_batch.cpu().detach().numpy().reshape(-1)))
        
    val_loss = np.mean(val_losses)
    val_accuracy = np.mean(val_accs)
    
    print(f"Epoch {epoch_num} / {epoch_count} | val Loss = {val_loss} | val accuracy = {val_accuracy}")

    
    # write losses and metrics to history lists
    train_loss_history.append(loss_value.item())
    val_loss_history.append(val_loss)
    val_acc_history.append(val_accuracy)
    
    
    # TODO: Сохранять лучшую модель по метрикам на валидации
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        best_model = model
    # TODO: Добавить early stopping: если модель на валидации не улучшается некоторое 
    # количество эпох, то прекратить обучение
    if val_loss_epoch[0] > min_val_loss and val_loss_epoch[1] > min_val_loss:
        break
    val_loss_epoch.append(val_loss)
    if len(val_loss_epoch) > 2:
        # fixed size of 2
        val_loss_epoch.pop(0)
    

    

In [ ]:
# TODO: Нарисовать графики изменения лосса на трейне/валидации в зависимсоти от эпох и аналогичный график по метрикам

In [54]:
train_loss_history

In [56]:
import matplotlib.pyplot as plt
plt.plot(train_loss_history)
plt.plot(val_loss_history)
plt.plot(val_acc_history)
plt.show()